[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jburgy/blog/blob/main/linprog/allocation_ampl.ipynb)

In [ ]:
%pip install amplpy ampl-module-base ampl-module-highs --extra-index-url https://pypi.ampl.com

In [ ]:
import numpy as np
import pandas as pd
from amplpy import ampl_notebook
from google.colab import userdata

ampl = ampl_notebook(license_uuid=userdata.get("AMPL_CE_LICENSE"))

In [ ]:
%%ampl_eval
set STOCKS;
set ACCOUNTS = {1..2};
set SECTORS;
# see https://groups.google.com/g/ampl/c/ULGck_3EQOM/m/yHvDLokzBQAJ
set STOCKS_IN_SECTOR {SECTORS} within STOCKS;

param base {ACCOUNTS};
param market_value {STOCKS};
param sign {s in STOCKS} = if market_value[s] >= 0 then 1 else -1;
param volume {STOCKS};
param conc_size {ACCOUNTS} default 0 >= 0;
set ACCOUNTS_WITH_CONC = {a in ACCOUNTS: conc_size[a]>0};
param conc_break {a in ACCOUNTS_WITH_CONC, p in 1..conc_size[a]}
          >= if p = 1 then 0.0 else conc_break[a, p-1];
param conc_rate {a in ACCOUNTS_WITH_CONC, p in 1..conc_size[a]}
          > if p = 1 then 0.0 else conc_rate[a, p-1];
param sect_size {ACCOUNTS} default 0 >= 0;
set ACCOUNTS_WITH_SECT = {a in ACCOUNTS: conc_size[a]>0};
param sect_break {a in ACCOUNTS_WITH_SECT, p in 1..conc_size[a]}
          >= if p = 1 then 0.0 else sect_break[a, p-1];
param sect_rate {a in ACCOUNTS_WITH_SECT, p in 1..conc_size[a]}
          > if p = 1 then 0.0 else sect_rate[a, p-1];

minimize TotalCost;

subject to Completeness {s in STOCKS}:
          to_come = market_value[s];

var X {s in STOCKS, ACCOUNTS}
          >= <<0; 1,0>> market_value[s], <= <<0; 0,1>> market_value[s],
          coeff Completeness[s] 1;

subject to AbsXLevel {s in STOCKS, a in ACCOUNTS}:
          to_come = sign[s] * X[s, a];

var AbsX {s in STOCKS, a in ACCOUNTS} >= 0,
          obj TotalCost base[a],
          coeff AbsXLevel[s, a] 1;

subject to GMVLevel {a in ACCOUNTS}:
          to_come = sum {s in STOCKS} AbsX[s, a];

var GMV {a in ACCOUNTS} >= 0,
          coeff GMVLevel[a] 1;

subject to ConcentrationLevel {s in STOCKS, a in ACCOUNTS_WITH_CONC}:
          to_come = AbsX[s, a] - 0.05 * GMV[a];

subject to SectorMVLevel {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come = sum {s in STOCKS_IN_SECTOR[sector]} X[s, a];

var SectorMV {sector in SECTORS, a in ACCOUNTS_WITH_SECT},
          coeff SectorMVLevel[sector, a] 1;

subject to SectorNMVDomain {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come >= <<0; -1,1>> SectorMV[sector, a];

var SectorNMV {sector in SECTORS, a in ACCOUNTS_WITH_SECT} >= 0,
          coeff SectorNMVDomain[sector, a] 1;

subject to SectorExcessLevel {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come = SectorNMV[sector, a] - 0.2 * GMV[a];

var Concentration {s in STOCKS, a in ACCOUNTS_WITH_CONC},
          coeff ConcentrationLevel[s, a] 1;

subject to ConcentrationPenaltyLevel {s in STOCKS, a in ACCOUNTS_WITH_CONC}:
          to_come >= <<{p in 1..conc_size[a]} conc_break[a, p];
                       {p in 0..conc_size[a]} if p = 0 then 0.0 else conc_rate[a, p]>> Concentration[s, a];

var ConcentrationPenalty {s in STOCKS, a in ACCOUNTS_WITH_CONC},
          obj TotalCost 1,
          coeff ConcentrationPenaltyLevel[s, a] 1;

var SectorExcess {sector in SECTORS, a in ACCOUNTS_WITH_SECT},
          coeff SectorExcessLevel[sector, a] 1;

subject to SectorPenaltyLevel {sector in SECTORS, a in ACCOUNTS_WITH_SECT}:
          to_come >= <<{p in 1..sect_size[a]} sect_break[a, p];
                       {p in 0..sect_size[a]} if p = 0 then 0.0 else sect_rate[a, p]>> SectorExcess[sector, a];

var SectorPenalty {sector in SECTORS, a in ACCOUNTS_WITH_SECT},
          obj TotalCost 1,
          coeff SectorPenaltyLevel[sector, a] 1;

In [ ]:
penalties = pd.DataFrame(
    [["conc", 1, 1, 0.0, 0.1], ["sect", 1, 1, 0.0, 0.1]],
    columns=["name", "account", "index", "break", "rate"],
).set_index(["name", "account", "index"])
penalties

In [ ]:
π = pd.read_csv(
    "https://raw.githubusercontent.com/jburgy/jupyter"
    "/refs/heads/main/content/data/portfolio.csv",
    index_col=0,
)
# Fill-in blank ticker (ESC GCI LIBERTY INC SR COMMON STOCK)
π.rename(index={np.nan: "DUMMY"}, inplace=True)
π["market_value"] = π["Shares"] * π["Price"]
π["volume"] = π["Volume"] * π["Price"]

ampl.set_data(π[["market_value", "volume"]], set_name="STOCKS")
ampl.param["base"] = {1: 0.05, 2: 0.06}
for name, p in penalties.groupby("name", as_index=False):
    ampl.param[f"{name}_size"] = p["break"].groupby("account").count().to_dict()
    ampl.param[f"{name}_break"] = p.loc[name, "break"].to_dict()
    ampl.param[f"{name}_rate"] = p.loc[name, "rate"].to_dict()

stocks_in_sector = π.groupby("Sector").groups
ampl.set["SECTORS"] = stocks_in_sector.keys()
ampl.set["STOCKS_IN_SECTOR"] = stocks_in_sector

ampl.option["solver"] = "highs"
ampl.option["highs_options"] = {"outlev": 1}
ampl.solve()
assert ampl.solve_result == "solved"

In [ ]:
X = ampl.get_variable("X").get_values().to_pandas()
π.assign(
    account_1=X.loc[(π.index, 1), "X.val"].to_numpy(),
    account_2=X.loc[(π.index, 2), "X.val"].to_numpy(),
)